In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet("hf://datasets/data-is-better-together/10k_prompts_ranked/data/train-00000-of-00001.parquet")

/home/dimitri/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = df.prompt

In [4]:
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

nltk.download('punkt')

data = df.prompt

sentences = data.apply(lambda x: word_tokenize(x.lower())).tolist()

model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)


[nltk_data] Downloading package punkt to /home/dimitri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
test_data  = pd.Series(["What"])

# Tokenisation de chaque phrase en minuscules
test_sentences = test_data.apply(lambda x: word_tokenize(x.lower())).tolist()

# Fonction pour calculer un vecteur moyen pour une phrase
def sentence_vector(sentence, model):
    vectors = [model.wv[word] for word in sentence if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    return None

# Calculer le vecteur moyen pour chaque phrase du dataset Test
test_vectors = [sentence_vector(sentence, model) for sentence in test_sentences]

In [7]:
def decode_embedding(embedding, model, topn=10):
    return model.wv.similar_by_vector(embedding, topn=topn)

# Décoder le vecteur moyen de la première phrase du dataset Test
decoded_words = decode_embedding(test_vectors[0], model)

# Affichage des mots les plus similaires
print("Mots les plus similaires au vecteur de la phrase Test :", decoded_words)

Mots les plus similaires au vecteur de la phrase Test : [('what', 1.0000001192092896), ('why', 0.8366110920906067), ('confluence', 0.7470139861106873), ('important', 0.7399217486381531), ('where', 0.7372153997421265), ('how', 0.7268548607826233), ('here', 0.719048023223877), ('benchmarked', 0.6970731616020203), ('perceive', 0.6880468726158142), ('so…my', 0.6844384670257568)]


In [22]:
def synonyms(word, model, topn=10):
    """
    Find synonyms for a given word using the Word2Vec model.
    """
    list_syno = decode_embedding(model.wv[word], model, topn)
    return list_syno[1][0]
def sentence_synonyms(sentence, model, topn=10):
    """
    Find synonyms for each word in a sentence using the Word2Vec model.
    """
    res = []
    sentence = sentence.lower().split()
    for word in sentence:
        if word in model.wv:
            res.append(synonyms(word, model, topn))
        else:
            res.append(word)
    return res

In [26]:
sentence_synonyms("how old are you", model, 10)

['what', 'ago', 'were', 'we']